In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC, LinearSVC # SVCは非線形SVM（Support Vector Classifier）
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# ファイル入出力ライブラリ
import pickle

# データの読み込み

In [2]:
df_input = pd.read_csv('../bank_marketing_train.csv')

# 特徴量エンジニアリング

### 最終的なペルソナに従い、特徴量を抽出する
- age:60以上
- job:retired
- marital：結婚経験あり
- default(クレジットの支払い遅延)：なし
- education(最終学歴)：basic.4y
- contact(連絡デバイス)：cellular
- pdays（前回の接触からの経過日数）：少ない
- poutcome（以前のキャンペーン結果）：あり（初めての客でない）
- previous(以前のキャンペーンの接触回数)：数が多い（頻繁に連絡をとっている）

### ペルソナに用いた説明変数以外で重要なものも含める
- emp.var.rate
- cons.price.idx
- cons.conf.idx

In [3]:
df_extract = df_input.loc[:, ['age','job','marital','default','education','contact','pdays','poutcome','previous',
                            'emp.var.rate','cons.price.idx','cons.conf.idx','y']]
# df_extract = df_input.loc[:, ['age','job','marital','default','education','housing','loan','contact','pdays','poutcome',
#                             'previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','y']]

In [4]:
df_extract.head(2)

,age,job,marital,default,education,contact,pdays,poutcome,previous,emp.var.rate,cons.price.idx,cons.conf.idx,y
0,56,housemaid,married,no,basic.4y,telephone,999,nonexistent,0,1.1,93.994,-36.4,no
1,57,services,married,unknown,high.school,telephone,999,nonexistent,0,1.1,93.994,-36.4,no


### 特徴量と目的変数に分ける

In [5]:
df_X = df_extract.drop('y', axis=1)

In [6]:
df_X.head(2)

,age,job,marital,default,education,contact,pdays,poutcome,previous,emp.var.rate,cons.price.idx,cons.conf.idx
0,56,housemaid,married,no,basic.4y,telephone,999,nonexistent,0,1.1,93.994,-36.4
1,57,services,married,unknown,high.school,telephone,999,nonexistent,0,1.1,93.994,-36.4


In [7]:
df_y = df_extract['y']

In [8]:
df_y.head(2)

0    no
1    no
Name: y, dtype: object

### 目的変数のyes/noを1/0に変換

In [9]:
df_y_val = df_y.apply(lambda x: 1 if x == 'yes' else 0)

In [10]:
df_y_val.head(2)

0    0
1    0
Name: y, dtype: int64

### Dummyエンコーディング

In [14]:
# カテゴリカル変数
categorical_columns = ['job','marital','default','education','contact','poutcome']

In [15]:
df_X_dm = pd.get_dummies(data=df_X, dummy_na=True, drop_first=True, columns=categorical_columns)

In [16]:
df_X_dm.columns

Index(['age', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'job_nan', 'marital_married', 'marital_single', 'marital_unknown',
       'marital_nan', 'default_unknown', 'default_yes', 'default_nan',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'education_nan',
       'contact_telephone', 'contact_nan', 'poutcome_nonexistent',
       'poutcome_success', 'poutcome_nan'],
      dtype='object')

### 機械学習モデルに入力するためNumpy配列に変換

In [17]:
y = np.array(df_y_val)
X = np.array(df_X_dm)

In [18]:
print(y)
print(X)

[0 0 0 ..., 1 1 0]
[[  56.  999.    0. ...,    1.    0.    0.]
 [  57.  999.    0. ...,    1.    0.    0.]
 [  56.  999.    0. ...,    1.    0.    0.]
 ..., 
 [  73.  999.    0. ...,    1.    0.    0.]
 [  44.  999.    0. ...,    1.    0.    0.]
 [  74.  999.    1. ...,    0.    0.    0.]]


### HoldOut

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(df_X_dm, df_y_val, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Modelの生成

In [20]:
logistic = Pipeline([('scl',StandardScaler()), ('est',LogisticRegression(random_state=1, class_weight="balanced"))])
# random_forest = Pipeline([('scl',StandardScaler()), ('est',RandomForestClassifier(random_state=1, class_weight="balanced"))])
# xgb = Pipeline([('scl',StandardScaler()), ('est',XGBClassifier(random_state=1, class_weight="balanced"))])
# lgbm = Pipeline([('scl',StandardScaler()), ('est',LGBMClassifier(random_state=1, class_weight="balanced"))])

# 学習

In [21]:
logistic.fit(X_train, y_train)
# random_forest.fit(X_train, y_train)
# xgb.fit(X_train, y_train)
# lgbm.fit(X_train, y_train)

C:\Users\riode\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

# 予測

In [22]:
y_pred_proba = logistic.predict_proba(X_test)
# y_pred_proba = random_forest.predict_proba(X_test)
# y_pred_proba = xgb.predict_proba(X_test)
# y_pred_proba = lgbm.predict_proba(X_test)

# ベストな閾値を求める

### ROIの計算処理

In [23]:
def calc_roi(y_list: np.ndarray, attack_list: np.ndarray) -> int:
    roi = 0
    for (yes, attack) in zip(y_list, attack_list):
    #     print(yes, attack)
        if attack == 1:
            roi += 5000*yes - 500*attack

    return roi

### 閾値を変えてROIを計算

In [24]:
best_roi = 0
best_threshold = 0
best_attack_list = np.ndarray([])

for threshold in np.arange(0, 1.01, 0.01):
    attack_list = (y_pred_proba[:, 1] > threshold).astype(int)
    roi = calc_roi(y_test, attack_list)
#     print('threshold:', threshold, ', roi:', roi)
    if roi > best_roi:
        best_roi = roi
        best_threshold = threshold
        best_attack_list = attack_list

In [25]:
print(best_roi)
print(best_threshold)
print(best_attack_list)

2329000
0.53
[0 0 1 ..., 0 0 1]


### ベストな閾値は上の通り

# 複数モデルの中からベストモデルを選ぶ

### 複数モデルを構築

In [26]:
pipelines = {
            'knn':
                Pipeline([('scl',StandardScaler()), ('est',KNeighborsClassifier())]),
            'logistic':
                Pipeline([('scl',StandardScaler()), ('est',LogisticRegression(random_state=1, class_weight="balanced"))]),
            'rsvc':
                Pipeline([('scl',StandardScaler()), ('est',SVC(C=1.0,kernel='rbf',class_weight='balanced',random_state=1, probability=True))]),
#             'lsvc':
#                 Pipeline([('scl',StandardScaler()), ('est',LinearSVC(C=1.0,random_state=1,class_weight='balanced'))]),
            'rf':
                Pipeline([('scl',StandardScaler()), ('est',RandomForestClassifier(random_state=1, class_weight="balanced"))]),
            'gb':
                Pipeline([('scl',StandardScaler()), ('est',GradientBoostingClassifier(random_state=1))]),
            'mlp':
                Pipeline([('scl',StandardScaler()), ('est',MLPClassifier(hidden_layer_sizes=(5,3), max_iter=500, random_state=1))]),
            'xgb':
                Pipeline([('scl',StandardScaler()), ('est',XGBClassifier(random_state=1, class_weight="balanced"))]),
            'lgbm':
                Pipeline([('scl',StandardScaler()), ('est',LGBMClassifier(random_state=1, class_weight="balanced"))]),
    }

### 各モデルのROIを求める

In [27]:
model_info = {}
for pipe_name, pipeline in pipelines.items():
    # 学習
    pipeline.fit(X_train, y_train)
    # 予測
    y_pred_proba = pipeline.predict_proba(X_test)
    
    best_roi = 0
    best_threshold = 0
    best_attack_list = np.ndarray([])

    for threshold in np.arange(0, 1.01, 0.01):
        attack_list = (y_pred_proba[:, 1] > threshold).astype(int)
        roi = calc_roi(y_test, attack_list)
    #     print('threshold:', threshold, ', roi:', roi)
        if roi > best_roi:
            best_roi = roi
            best_threshold = threshold
            best_attack_list = attack_list
    
    model_info[pipe_name] = {'model': pipeline, 'roi': best_roi, 'threshold': best_threshold, 'attack_list': best_attack_list}

C:\Users\riode\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\riode\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\riode\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### # 各モデルのベストROIと閾値を確認

In [28]:
for model_name, info in model_info.items():
    print(model_name)
    print(' roi       :', info['roi'])
    print(' threshold :', info['threshold'])

knn
 roi       : 1738000
 threshold : 0.2
logistic
 roi       : 2329000
 threshold : 0.53
rsvc
 roi       : 2333000
 threshold : 0.15
rf
 roi       : 1807500
 threshold : 0.19
gb
 roi       : 2484000
 threshold : 0.13
mlp
 roi       : 2354000
 threshold : 0.11
xgb
 roi       : 2482000
 threshold : 0.13
lgbm
 roi       : 2439500
 threshold : 0.54


# アタックリストの作成

In [33]:
def make_attack_list(dataset: pd.DataFrame, model: Pipeline, threshold: float) -> np.ndarray:

    # ペルソナに用いた説明変数、重要な説明変数を含める(yは含めないように注意)
    df_X = dataset.loc[:, ['age','job','marital','default','education','contact','pdays','poutcome','previous',
                            'emp.var.rate','cons.price.idx','cons.conf.idx']]
    
    # Dummyエンコーディング
    categorical_columns = ['job','marital','default','education','contact','poutcome']
    df_X_dm = pd.get_dummies(data=df_X, dummy_na=True, drop_first=True, columns=categorical_columns)
    
    # 機械学習モデルに入力するためNumpy配列に変換
    X = np.array(df_X_dm)
    
    # Modelの生成
    y_pred_proba = model.predict_proba(X)
    
    # アタックリストの作成
    attack_list = (y_pred_proba[:, 1] > threshold).astype(int)
    
    return attack_list

In [34]:
# 当日はテストデータを読み込む
df_test = pd.read_csv('../bank_marketing_train.csv')
final_attack_list = make_attack_list(df_test, model_info['xgb']['model'], model_info['xgb']['threshold'])

In [35]:
final_attack_list

array([0, 0, 0, ..., 1, 1, 1])

### アタックリストをcsv出力

In [36]:
np.savetxt('attack.csv', final_attack_list,delimiter=',', fmt='%d')

# 期待される収益

In [37]:
# yのリストを求める
y = df_test['y']
y_val = y.apply(lambda x: 1 if x == 'yes' else 0)
y_list = np.array(y_val)

In [38]:
# ROIの計算
calc_roi(y_list=y_list, attack_list=final_attack_list)

8597500

### （おまけ）yをすべて当てた時の理想収益

In [39]:
y_val[y_val == 1].size * (5000-500)

17082000

In [42]:
# 予測を外した件数
(17082000-8597500)/500

16969.0

# Modelの予測精度

In [43]:
print('accuracy  :', accuracy_score(y_true=y_list, y_pred=final_attack_list))
print('recall    :', recall_score(y_true=y_list, y_pred=final_attack_list))
print('precision :', precision_score(y_true=y_list, y_pred=final_attack_list))
print('f1        :', f1_score(y_true=y_list, y_pred=final_attack_list))

accuracy  : 0.843972261735
recall    : 0.6145943098
precision : 0.380277098615
f1        : 0.469841909173
